In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于批量预测的自动ML表格预测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建表格预测模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测。

### 数据集

本教程使用的数据集是BigQuery公共数据集，[《纽约时报美国冠状病毒数据库》](https://pantheon.corp.google.com/marketplace/product/the-new-york-times/covid19_us_cases)。该数据集不需要任何特征工程。您在本教程中将使用的数据集版本存储在一个公共的Cloud Storage存储桶中。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML表格预测模型，然后使用Vertex客户端库进行批量预测。您还可以选择使用`gcloud`命令行工具或在Google Cloud Console上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

在使用批量预测和在线预测之间有一个关键区别：

* 预测服务：为整个实例集（即一个或多个数据项）执行按需预测，并实时返回结果。

* 批量预测服务：为整个实例集进行排队（批处理）预测，后台处理并在准备就绪时将结果存储在Cloud Storage存储桶中。

### 成本

本教程使用 Google Cloud (GCP) 的可计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的google-cloud-storage库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

### GPU运行时

*如果有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个帐户时，您将获得$300的免费信用用于您的计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk)已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在命令中使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 `REGION` 变量，该变量用于本笔记本其余部分的操作。以下是支持 Vertex 的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法在 Vertex 中使用多区域存储桶进行培训。并非所有区域都支持所有 Vertex 服务。有关每个区域的最新支持，请参阅[Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐户**。

在**服务帐户名称**字段中，输入一个名称，然后单击** 创建**。

在**授予此服务帐户对项目的访问权限**部分，单击角色下拉列表。在过滤框中键入“Vertex”，并选择**Vertex管理员**。在过滤框中键入“存储对象管理员”，并选择**存储对象管理员**。

单击创建。包含您的密钥的JSON文件将下载到您的本地环境。

将您的服务帐户密钥的路径输入到下面的单元格中作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**以下步骤都是必需的，无论您使用的是哪种笔记本环境。**

本教程旨在使用存储在公共云存储桶中的训练数据以及本地云存储桶进行批量预测。您也可以使用在本地云存储桶中存储的自己的训练数据。

请在下方设置您的云存储桶名称。存储桶名称必须在所有谷歌云项目中是全局唯一的，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才运行以下单元格创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和端点服务端点。
- `PARENT`：数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常量

设置专属于AutoML数据集和训练的常量：

- 数据集模式：告诉“数据集”资源服务数据集的类型是什么。
- 数据标记（注释）模式：告诉“数据集”资源服务数据如何标记（注释）。
- 数据集训练模式：告诉“Pipeline”资源服务为哪个任务（例如分类）训练模型。

In [ ]:
# Forecasting Dataset type
DATA_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/metadata/time_series_1.0.0.yaml"
)
# Forecasting Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_forecasting_1.0.0.yaml"

硬件加速器

设置用于预测的硬件加速器（例如GPU）。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个包含4个Nvidia Telsa K80 GPU的GPU容器映像分配给每个VM，您需要指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80, 4）

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### 容器（Docker）镜像

对于自动机器学习的批量预测，服务二进制的容器镜像是由Vertex预测服务预先确定的。更具体地说，该服务将根据您选择的硬件加速器为模型选择适当的容器。

#### 机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置将用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始创建自己的AutoML表格预测模型了。

## 设置客户端

Vertex 客户端库以客户端/服务器模型运行。在您的一侧（Python 脚本），您将创建一个客户端，该客户端向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 数据集服务用于`数据集`资源。
- 模型服务用于`模型`资源。
- 用于训练的流水线服务。
- 用于批量预测和自定义训练的作业服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

数据准备

对于表格数据，Vertex `Dataset` 资源有一些要求。

- 必须是 CSV 文件或 BigQuery 查询。

#### CSV

对于表格预测来说，CSV文件有一些要求：

- 第一行必须是标题行 -- 注意这与 Vision、Video 和 Language 不同，它们的要求是没有标题行。
- 除了一个列是特征列外，其他列都是特征列。
- 一个列是标签列，你将在随后创建训练管道时指定。
- 一个列是时间列，你将在随后创建训练管道时指定。
- 一个列是时间序列标识符列，你将在随后创建训练管道时指定。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为Cloud Storage中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/ai-platform/covid/bigquery-public-covid-nyt-us-counties-train.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的NY Times COVID数据库数据集的版本，使用CSV索引文件。

首先，快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

您还需要知道训练时标签列的标题名称，我们将其保存为`label_column`。此外，为了识别文件中的时间序列，训练需要知道另外两列，`time_column`提供时间信息，`time_series_identifier_column`中的单元格具有相同值构成一个时间序列。对于这个数据集，标签列是`deaths`，时间列是`date`，时间序列标识列是`county`。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = "deaths"  # @param {type:"string"}
time_column = "date"  # @param {type:"string"}
time_series_identifier_column = "county"  # @param {type:"string"}
print("Label Column Name", label_column)
print("Time Column Name", time_column)
print("Time Series Identifier Column Name", time_series_identifier_column)

## 数据集

现在您的客户准备就绪，训练模型的第一步是创建一个托管的数据集实例，然后将您的标记数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建 `Dataset` 资源的实例。此函数执行以下操作：

1. 使用数据集客户服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的可读名称。
 - `metadata_schema_uri`：数据集类型的模式。
 - `metadata`：表格数据的云存储或BigQuery位置。
3. 调用客户数据集服务方法 `create_dataset`，具有以下参数：
 - `parent`：您的 `Database`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
 - `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法将返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用的方式。尽管这一步通常很快，当您在项目中首次使用时，由于设置操作，可能会有较长的延迟。

您可以使用 `operation` 对象来获取操作的状态（例如创建 `Dataset` 资源）或取消操作，通过调用操作方法：

| 方法        | 描述 |
| ----------- | ----------- |
| result()    | 等待操作完成并以 JSON 格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的 True/False。        |
| done()      | 返回操作是否已完成的 True/False。 |
| canceled()  | 返回操作是否已取消的 True/False。 |
| cancel()    | 取消操作（可能需要最多 30 秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, src_uri=None, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        if src_uri.startswith("gs://"):
            metadata = {"input_config": {"gcs_source": {"uri": [src_uri]}}}
        elif src_uri.startswith("bq://"):
            metadata = {"input_config": {"bigquery_source": {"uri": [src_uri]}}}
        dataset = aip.Dataset(
            display_name=name,
            metadata_schema_uri=schema,
            labels=labels,
            metadata=json_format.ParseDict(metadata, Value()),
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("covid-" + TIMESTAMP, DATA_SCHEMA, src_uri=IMPORT_FILE)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

训练模型

现在使用您的 Vertex `Dataset` 资源训练一个 AutoML 表格预测模型。要训练模型，请按照以下步骤操作：

1. 为 `Dataset` 资源创建一个 Vertex 训练管道。
2. 执行管道开始训练。

### 创建一个训练管道

您可能会问，我们用管道做什么？通常在工作（如训练）有多个步骤时，我们会按照顺序进行：执行步骤A，执行步骤B，依此类推。将这些步骤放入管道中，我们将获得以下好处：

1. 可以重复用于后续的训练工作。
2. 可以封装为容器并作为批处理作业运行。
3. 可以进行分布式处理。
4. 所有步骤都与同一管道作业相关联，以跟踪进度。

使用这个辅助函数`create_pipeline`，它接受以下参数：

- `pipeline_name`：管道作业的易读名称。
- `model_name`：模型的易读名称。
- `dataset`：Vertex 完全合格的数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练作业需求的字典。

这个辅助函数调用`Pipeline`客户端服务的`create_pipeline`方法，它接受以下参数：

- `parent`：您的`数据集`、`模型`和`端点`资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规范。

现在让我们深入了解构建`training_pipeline`规范的*最小*要求：

- `display_name`：管道作业的易读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练作业需求的字典。
- `model_to_upload`：模型的易读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅 Vertex 数据集标识符（非完全合格）-- 这是完全合格标识符的最后一部分。
 - `fraction_split`：如果指定，则用于训练、测试和验证的数据集的百分比。否则，百分比会自动由 AutoML 选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，需要一个 Google protobuf 结构 (类似于 Python 字典) 而不是 Python (JSON-like) 字典的 `task` 字段。使用 `json_format.ParseDict` 方法进行转换。

您需要指定的最小字段包括：

- `target_column`：我们要预测的 CSV 表头列的列名（即标签）。
- `time_column`：表示时间序列的时间信息的 CSV 表头列名。
- `time_series_identifier_column`：用于标识单个时间序列的列的 CSV 表头列名，例如，此列的值组成一个时间序列的行。
- `period`：同一时间序列中两次观察之间的时间间隔，例如，每小时、每日等。
- `forecast_window_end`：指示未来要预测多少个 `period` 的数字。
- `time_variant_past_only_columns`：在预测时未知的列的 CSV 表头列名（即标签）。
- `train_budget_milli_node_hours`：训练模型的最大时间预算，其中 1000 = 1 小时。
- `disable_early_stopping`：是 True 还是 False，用于让 AutoML 根据情况决定是否早停训练或完全训练。
- `transformations`：为每个特征列指定特征工程。

对于 `transformations`，列表必须为每列指定一个条目。外部键字段表示相应列的特征工程类型。在本教程中，将其设置为`"auto"`，以告诉 AutoML 自动确定。

最后，通过调用帮助函数 `create_pipeline` 来创建管道，该函数将返回一个训练管道对象的实例。

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "date"}},
    {"auto": {"column_name": "state_name"}},
    {"auto": {"column_name": "county_fips_code"}},
    {"auto": {"column_name": "confirmed_cases"}},
    {"auto": {"column_name": "deaths"}},
]

PERIOD = {"unit": "day", "quantity": 1}

PAST_ONLY_COLUMNS = ["deaths"]
STATIC_COLUMNS = ["state_name"]
PAST_AND_FUTURE_COLUMNS = ["date"]

optimization_objective = "minimize-rmse"

In [ ]:
PIPE_NAME = "covid_pipe-" + TIMESTAMP
MODEL_NAME = "covid_model-" + TIMESTAMP

task = Value(
    struct_value=Struct(
        fields={
            "target_column": Value(string_value=label_column),
            "time_column": Value(string_value=time_column),
            "time_series_identifier_column": Value(
                string_value=time_series_identifier_column
            ),
            "period": json_format.ParseDict(PERIOD, Value()),
            "forecast_window_end": Value(number_value=10),
            "time_variant_past_only_columns": json_format.ParseDict(
                PAST_ONLY_COLUMNS, Value()
            ),
            "static_columns": json_format.ParseDict(STATIC_COLUMNS, Value()),
            "time_variant_past_and_future_columns": json_format.ParseDict(
                PAST_AND_FUTURE_COLUMNS, Value()
            ),
            "train_budget_milli_node_hours": Value(number_value=1000),
            "optimization_objective": Value(string_value=optimization_objective),
            "transformations": json_format.ParseDict(TRANSFORMATIONS, Value()),
        }
    )
)
response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取关于培训流水线的信息

现在仅获取此培训流水线实例的流水线信息。通过调用作业客户端服务的`get_training_pipeline`方法，使用以下参数获取此作业的作业信息：

- `name`：Vertex的完全限定流水线标识符。

当模型训练完成时，流水线状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练上述模型可能需要长达60分钟。

一旦您的模型训练完成，您可以通过从 `end_time` 减去 `start_time` 来计算训练模型所花费的实际时间。对于您的模型，您需要知道完全限定的Vertex模型资源标识符，即管道服务分配给它的。您可以从返回的管道实例中获取此信息，作为字段 `model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在你的模型已经训练完成，你可以获取一些关于你的模型的信息。

## 评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，一部分数据集被保留为测试（留出）数据，由管道服务用于评估模型。

### 列出所有切片的评估

使用这个辅助函数 `list_model_evaluations`，它接受以下参数：

- `name`: Vertex 中的完全合格的 `Model` 资源标识符。

这个辅助函数使用 model client 服务的 `list_model_evaluations` 方法，接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 你可能只有一个，然后我们打印出每个评估中的所有关键名称，对于一个小集合（`rootMeanSquaredError` 和 `meanAbsoluteError`），你将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("rootMeanSquaredError", metrics["rootMeanSquaredError"])
        print("meanAbsoluteError", metrics["meanAbsoluteError"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 批量预测的模型部署

现在部署你为批量预测创建的已训练的 Vertex `Model` 资源。这与为按需预测部署 `Model` 资源不同。

对于在线预测，你需要：

1. 创建一个 `Endpoint` 资源来部署 `Model` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

3. 发送在线预测请求到 `Endpoint` 资源。

对于批量预测，你需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求配置资源。

3. 将批量预测请求的结果返回给调用方。

4. 作业服务将取消为批量预测请求配置的资源。

进行一个批量预测请求

现在对您部署的模型进行批量预测。

制作测试项目

您将使用合成数据作为测试数据项目。不要担心我们正在使用合成数据 - 我们只是想演示如何进行预测。

In [ ]:
HEADING = "date,county,state_name,county_fips_code,confirmed_cases,deaths"
INSTANCE_1 = "2020-10-13,Adair,Iowa,19001,103,null"
INSTANCE_2 = "2020-10-29,Adair,Iowa,19001,197,null"

创建批量输入文件

现在创建一个批量输入文件，您将把它存储在您本地的云存储桶中。与图像、视频和文本不同，表格的批量输入文件只支持CSV格式。对于CSV文件，您需要做如下操作：

- 第一行是带有特征（字段）标题名称的标题行。
- 每个剩余行是一个单独的预测请求，带有相应的特征数值。

例如：

    "特征_1", "特征_2". ...
    值_1, 值_2, ...

In [ ]:
import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.csv"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    f.write(HEADING + "\n")
    f.write(str(INSTANCE_1) + "\n")
    f.write(str(INSTANCE_2) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例扩展

在处理批量预测请求时，您有几种选择可以对计算实例进行扩展：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：批量预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，将提供固定数量的计算实例并将批量预测请求均匀分布在它们之间。

- 自动扩展：批量预测请求在可扩展数量的计算实例之间分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为首次部署模型时提供和取消提供的计算实例数，并将最大（`MAX_NODES`）数量设置为根据负载情况提供的计算实例数。

最小计算实例数量对应于字段 `min_replica_count`，最大计算实例数量对应于字段 `max_replica_count`，在您随后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 发起批量预测请求

现在您的两个测试项的批量已经准备好了，让我们进行批量请求。使用以下参数调用这个帮助函数`create_batch_prediction_job`：

- `display_name`：预测任务的人类可读名称。
- `model_name`：`Model`资源的Vertex完全限定标识符。
- `gcs_source_uri`：输入文件的云存储路径--您之前创建的。
- `gcs_destination_output_uri_prefix`：服务将写入预测结果的云存储路径。
- `parameters`：用于提供预测结果的附加过滤参数。

帮助函数调用作业客户端服务的`create_batch_prediction_job`方法，使用以下参数：

- `parent`：Dataset、Model和Pipeline资源的Vertex根路径。
- `batch_prediction_job`：批量预测任务的规范。

现在让我们深入研究`batch_prediction_job`的规范：

- `display_name`：预测批处理作业的人类可读名称。
- `model`：`Model`资源的Vertex完全限定标识符。
- `dedicated_resources`：为批量预测任务分配的计算资源。
  - `machine_spec`：要分配的计算实例。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。
  - `starting_replica_count`：初始准备的计算实例数，您之前设置为变量`MIN_NODES`。
  - `max_replica_count`：可扩展的最大计算实例数，您之前设置为变量`MAX_NODES`。
- `model_parameters`：用于提供预测结果的附加过滤参数。*注意*，图像分割模型不支持额外参数。
- `input_config`：用于预测实例的输入源和格式类型。
 - `instances_format`：批量预测请求文件的格式：仅支持`csv`。
 - `gcs_source`：您的批量预测请求的一个或多个云存储路径列表。
- `output_config`：预测结果的输出目的地和格式。
 - `prediction_format`：批量预测响应文件的格式：仅支持`csv`。
 - `gcs_destination`：预测的输出目的地。

这个调用是一个异步操作。您将从响应对象中打印一些选择字段，包括：

- `name`：分配给批量预测任务的Vertex完全限定标识符。
- `display_name`：预测批处理作业的人类可读名称。
- `model`：Model资源的Vertex完全限定标识符。
- `generate_explanations`：与预测一起提供的True/False解释（可解释性）。
- `state`：预测任务的状态（待定、运行中等）。

由于这个调用需要一些时间来执行，您可能会得到`JobState.JOB_STATE_PENDING`作为`state`。

In [ ]:
BATCH_MODEL = "covid_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "csv"
OUT_FORMAT = "csv"  # [csv]

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME, None
)

现在获取您创建的批处理预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用此辅助函数 `get_batch_prediction_job`，并使用以下参数：

- `job_name`：批量预测作业的 Vertex 完全合格标识符。

辅助函数调用作业客户端服务的 `get_batch_prediction_job` 方法，并使用以下参数：

- `name`：批量预测作业的 Vertex 完全合格标识符。在本教程中，您将向其传递批量预测作业的 Vertex 完全合格标识符 -- `batch_job_id`

辅助函数将返回存储预测结果的 Cloud 存储路径 -- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测

当批量预测处理完成时，作业状态将是`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在云存储路径中的预测结果，这是您设置为输出的位置。预测结果将以CSV格式显示，这是您在创建批处理预测作业时指定的，在以`prediction`开头的子文件夹中，文件夹中将有一个名为`predictions*.csv`的文件。

现在显示（cat）内容。您将看到多行数据，每行代表一个预测结果。

对于每个预测结果：

- 前四个字段是您进行预测的值（特征）。
- 剩余的字段是每个预测的置信度值，介于0和1之间。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction*.csv

        ! gsutil cat $folder/prediction*.csv
        break
    time.sleep(60)

清理工作

为了清理这个项目中使用的所有GCP资源，您可以删除用于本教程的GCP项目。

否则，您可以删除本教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME